### Libraries Imports

In [2]:
import pandas as pd
import functools
import numpy as np

import cufflinks as cf
import plotly.offline
import plotly.graph_objs as go
import plotly.plotly as py
import seaborn as sns
import matplotlib.pylab as plt
import plotly.offline
import plotly.graph_objs as go
import plotly.plotly as py
import math
import calendar
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 500)
pd.set_option('max_colwidth', 5000)
pd.set_option('display.float_format', '{:,}'.format)
pd.options.display.float_format = '{:,.0f}'.format
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)
py.sign_in('Zawar', 'nyd3v1Zb2R1iXqHDUuhU')
pd.options.mode.chained_assignment = None
from datetime import datetime
day_of_year = datetime.now().timetuple().tm_yday

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Load Premium , Vehcile & Claims Data

In [3]:
#DONOT DELETE   READING FROM DATABASE 
from datetime import datetime 

start_time = datetime.now() 
print(start_time)

import time

start_date_2015 = ['2019-01-01']
end_date_2015 = ['2019-12-31']

import pyodbc

conn = pyodbc.connect("DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={0}; database={1};UID={2};PWD={3}".format('172.16.2.86\SQL14','TDIIMSDAILY','bigdata','BIG!@#data'))
cursor = conn.cursor()
tempDf1 = pd.DataFrame()
for f, b in zip(start_date_2015, end_date_2015):
    print(f)
    storedProc = "EXEC [CLA_MotorIntimateReport_Current_DS_SP] @Company_Id=4, @ClaimStartDateFrom='"+str(f)+"', @EndTime='"+str(b)+"' "
    dfCurrent = pd.read_sql_query(storedProc, conn)
    dfCurrent = pd.concat([dfCurrent,tempDf1])
    tempDf1 = dfCurrent

print('Time elapsed (hh:mm:ss.ms) {}'.format(datetime.now() - start_time))


2020-04-21 14:40:44.538685
2019-01-01
Time elapsed (hh:mm:ss.ms) 0:13:44.205109


In [4]:
#tempDf1[(tempDf1.ClaimStage == 'Final Cost Estimation')]#[['ClaimTypes','InsuredAmount','GrossAmount','NetAmount','PaymentAmount','Amount_Reserve','PaidTo','EstSalvage','SalvageRecovery']]

df_claims = tempDf1.copy()
df_claims.shape

import pickle
with open('claimsRawData.pickle', 'wb') as f:
    pickle.dump(df_claims, f)


,BranchName,ClaimStage,ClientWise,LeadSource,CF_Id,COB_ID,isLeader,sm_ID,Activity_Id,PackageName,ClaimNo,IntimationDate,IncidentCity,IncidentDate,ContactName,NIC_No,Country,City,Area,Passport_No,SalesFormNo,BusinessType,ExpiryDate,EffectiveDate,InsuredAmount,GrossAmount,NetAmount,CircumstancesOfLoss,PaymentAmount,InvoiceAmount,WHTAmount,ClaimTypes,Stage_Id,CloseRemarks,Amount_Reserve,Reserve_GIS,Surveyour,LossController,Workshop,InsuranceType,RI_Amount,RI_Percent,RInsurer,Aging,CI_Amount,CI_Percent,CInsurer,TDI_Percent,TDI_Amount,GroupNo,Agent,PaidTo,VehicleMake,VehicleModel,VehicleColor,VehicleCategory,m_Registration_Year,m_Engine_No,m_Chasis_No,m_VehicleColor_Id,m_Registration_No,ChequeNo,Date,BankName,OutstandingPremium,SalvageRecovery,EstSalvage,Remarks,LossHandler,TaxRegion,claimstatus,WKRegion,WKType,WKMake,PitchRemarks,VehicleType,ClaimRegion,Account_Number,InternalAgent,LeadAgentName,SalesDate
0,Faisalabad,Claim Under Process,Corporate,Non 3S,163772,None,None,,57,None,FSD/P/WT/C/163582/0219,2019-02-02 15:28:03.950,Sahiwal,2019-02-02 14:30:00,M/S United Human Resource Services (Pvt) Ltd.,None,Pakistan,Rawalpindi,Dhoke Chaudhrian,None,ISB/P/WT/CUS043263/519097/0518,Renewal,2019-05-19,2018-05-20,"935,100","17,109","21,018","Incident Place & Current Location : Okara City\r\nThe ahead going vehicle suddenly stopped in the middle of the road. In order to avoid any collision, I immediately applied brakes but even then my car could not stop and hit against rear portion of that vehicle. At the same time a motorcyclist coming from behind also hit against left rear portion of my car.","31,375","4,140",nan,Accident,2,,0,,Tanvir-Ur-Rehman Pvt. Ltd.,None,Crystal Car Care--,None,nan,nan,None,444,nan,nan,None,nan,nan,None,,"31,375",Suzuki,Cultus VXR,Silky Silver,Private Motor Car,2015,PKF607672,SF410PK514528,"1,920",ED-867,,,,0,nan,0,,Mr. Rashid Amin,PRA,Claim Under Process,None,None,None,None,Local,CENTRAL,CUS043263,ATIF MEHMOOD,None,2018-05-05 12:11:05.870
1,Faisalabad,Claim Under Process,Corporate,Non 3S,164768,None,None,,57,None,FSD/P/TC/C/164571/0219,2019-02-18 09:59:56.357,Sargodha,2019-01-28 10:00:00,M/S HILTON PHARMA (PVT) LTD,42101-3397628-7,Pakistan,Karachi,Tariq Road,None,KHI/T/TC/CUS045371/010118/0219,Renewal,2019-12-31,2019-01-01,"708,000","18,530","21,124",hit by car,"17,760","7,100",nan,Accident,2,,0,,Tehseen Fazal Associates (Pvt) Ltd..,None,Suzuki Falcon Motors,None,nan,nan,None,428,nan,nan,None,nan,nan,None,,"17,760",Suzuki,Mehran VXR,Graphite Grey,Private Motor Car,2016,667821,1206352,"2,101",BGZ-648,,,,0,nan,0,,Mr. Rashid Amin,PRA,Claim Under Process,Suzuki Falcon Motors,3S,Suzuki,None,Local,CENTRAL,CUS045371,Adrian,Aon Insurance Broker (Pvt) ltd,2019-02-14 12:31:36.470
2,Faisalabad,Claim Under Process,Corporate,Non 3S,164819,None,None,,57,None,FSD/P/TC/C/164621/0219,2019-02-18 12:22:09.070,Sargodha,2019-02-08 17:00:00,M/S HILTON PHARMA (PVT) LTD,42101-3397628-7,Pakistan,Karachi,Tariq Road,None,KHI/T/TC/CUS045371/010125/0219,Renewal,2019-12-31,2019-01-01,"732,000","9,920","11,309",As per claim form we understand that a car struck with insured vehicle from left front and rear side.,"14,020",0,nan,Accident,2,,0,,Internal LossController,Mohammad Asim--,Suzuki Falcon Motors,None,nan,nan,None,428,nan,nan,None,nan,nan,None,,"14,020",Suzuki,Mehran,Graphite Grey,Private Motor Car,2017,709163,1247636,"2,101",BKZ-704,,,,0,nan,0,,Mr. Rashid Amin,PRA,Claim Under Process,Suzuki Falcon Motors,3S,Suzuki,None,Local,CENTRAL,CUS045371,Adrian,Aon Insurance Broker (Pvt) ltd,2019-02-14 17:45:24.047
3,Faisalabad,Claim Under Process,Corporate,Non 3S,166751,None,None,,57,None,FSD/P/WT/C/167005/0319,2019-03-20 13:58:58.947,Faisalabad,2019-03-19 19:00:00,M/S United Human Resource Services,None,Pakistan,Rawalpindi,Dhoke Chaudhrian,None,ISB/P/WT/CUS043263/504886/0418,Renewal,2019-04-23,2018-04-24,"950,000","17,383","21,355",hit with bike\r\nIncident Place * : sangla hill\r\nCurrent Location * sangla hill,"18,340",0,nan,Accident,2,,0,,Internal

In [6]:
#DONOT DELETE   READING FROM DATABASE 

from datetime import datetime 

start_time = datetime.now() 
print(start_time)

import time

start_date_2015 = ['2019-01-01']
end_date_2015 = ['2019-12-31']


import pyodbc

conn = pyodbc.connect("DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={0}; database={1};UID={2};PWD={3}".format('172.16.2.86\SQL14','TDIIMSDAILY','bigdata','BIG!@#data'))
cursor = conn.cursor()
tempDf2 = pd.DataFrame()
for f, b in zip(start_date_2015, end_date_2015):
    print(f, b)
    storedProc = "EXEC [SLS_GetVehicleWisePremiumRegister_DS_SP] @Company_Id1=4, @StartDate1='"+str(f)+"', @EndDate1='"+str(b)+"' , @Account_Type1=NULL "
    dfCurrent = pd.read_sql_query(storedProc, conn)
    dfCurrent = pd.concat([dfCurrent,tempDf2])
    tempDf2 = dfCurrent

print('Time elapsed (hh:mm:ss.ms) {}'.format(datetime.now() - start_time))

2020-04-21 15:49:54.863622
2019-01-01 2019-12-31
Time elapsed (hh:mm:ss.ms) 0:02:18.742938


In [7]:
tempDf2.rename(columns={'branch': 'SalesBranch','PolicyNo':'SalesFormNo'}, inplace=True)
df_prem = tempDf2.copy()
df_prem.VoucherType = np.where( (df_prem.VoucherType.str.contains("NB") == True), "Fresh","Renewal")


df_prem.head(2)

import pickle
with open('premRawData.pickle', 'wb') as f:
    pickle.dump(df_prem, f)

,Id,SalesformMapping_Id,row_number,groupid,Activity_Id,Cancellation_Type,SalesFormTaxesGroup_Id,Account_Type,CustomerType,Parent_SalesFormGroup_Id,Business,EndorsementNo,BillNo,BillDate,VoucherType,IsExistingCustomer,SalesFormNo,EffectiveDate,ExpiryDate,IssueDate,InsuredName,InsuredAmount,City,m_Registration_No,m_Engine_No,m_Chasis_No,VehicleMake,CommRate,CommAmount,IsLeadAgent,VehicleGrossAmount,ASCAmount,FEDAmount,FIFAmount,Adv_Tax,StampDuty,NetAmountT,NetRate,GrossRate,IntimationDate,ClaimNo,Reserve,SettleAmount,ClaimType,Showroom,LeadSource,InternalAgent,ParkingType,DriverType,m_CurrentMileage,Powered,Transmission_Type,CC,Color,Gender,CNIC,Address,Area,ContactCity,Occupation,HomeNo,OfficeNo,MobileNo,Status,AGE,SalesBranch,clpaid,clcount,CustomerSince,Product,PreRiskInspector,VehicleType,UW_Year,VehicleCategory,Outstanding,TransactionType_Id,Account_Number,Active/InActive,Account_Name,BasicPremium,LastActivity
0,1432554,2015843,55,1,57,None,1284988,I,Individual,"602,529",NB,,AR-MTR-PRM-3098953,2019-01-01,Renewal,No,KHI/P/PC/VA/CUS146690/656954/0119,2019-01-04,2020-01-03,2019-01-01,M/S BANK AL HABIB LTD A/C SYNERGIES SOURCING PAKISTAN (PVT) LTD USER: MR.YASSER MANZOOR QADIR QURESHI,"2,200,000",Karachi,BHG-425,2906435,009063,Honda Civic 2017,2,"1,434",Call Centre Commission (RENEWAL),"71,675","2,000","9,578",737,0,50,"83,990",4,3,2019-07-11 09:58:18.563,KHI/P/PC/C/174463/0719,0,"110,462",Accident,NONE,Non 3S,Syeda Sadia Nizami,None,None,nan,Patrol,Manual,1800,Black,,None,"ST-1/8, SECTOR 16,",Korangi Industrial Area,Karachi,-,None,None,None,Active,nan,Karachi,"110,462",2,4,Comprehensive Policy,None,Private Motor Car,2019,Local,0,4,CUS146690,InActive,Mr. BANK AL HABIB LTD. C/O. SYNERGIES SOURCING PAKISTAN (PVT) LTD USER: MR. BILAL USMAN,"51,484",Policy
1,1432623,2015987,56,1,57,None,1277466,I,Individual,"602,432",NB,,AR-MTR-PRM-3099072,2019-01-01,Renewal,No,KHI/P/PC/47078/656956/0119,2019-01-08,2020-01-07,2019-01-01,M/S RAZAQUE RAZNO TRADINGS (PVT) FATIMA FAZIL,"1,752,500",Karachi,BEX-387,Z386729,4060281,Toyota Corolla GLI-M/T 2016,1,219,Call Centre Commission (RENEWAL),"29,240","1,462","3,991",307,0,50,"35,000",2,2,NaT,None,nan,nan,None,NONE,Non 3S,Syeda Sadia Nizami,Bunglow,Self,nan,Patrol,Manual,1300,Strong Blue,,None,"PLOT NO. 22 7/D, K.D.A. SCHEME NO.1, KARACHI",Stadium Road,Karachi,-,None,None,None,Active,nan,Karachi,nan,0,9,Comprehensive Policy,None,Private Motor Car,2019,Local,0,4,47078,InActive,Mrs. M/S RAZAQUE RAZNO TRADINGS (PVT) USER FATIMA RIAZ FAZAL,"21,867",Policy


In [8]:
df_prem.VehicleGrossAmount.sum()

2109439279.0

In [ ]:
# df_prem = pd.read_csv('~/Projects/InsuranceAnalysis/data/SLS_PremiumRegister2014Onwards.csv',sep='|',engine='python')

# dict4Model = {}

# df_vehicle = pd.read_csv('~/Projects/InsuranceAnalysis/data/SLS_GetVehicleWisePremiumRegister_Fixed_2014Onwards.txt',sep='|',engine='python', usecols=range(70))

# df_claims = pd.read_csv('~/Projects/InsuranceAnalysis/data/CLA_MotorIntimateReport_2014Onwards.csv',sep='|')

In [ ]:
#df_prem[df_prem.VehicleCategory == 'Imported'].VehicleMake.unique().tolist()

## PreProcessing Data

In [9]:
df_claims['EffectiveDate'] =  pd.to_datetime(df_claims['EffectiveDate'],errors='coerce', format='%Y-%m-%d %H:%M:%S')
df_claims['IncidentDate'] =  pd.to_datetime(df_claims['IncidentDate'],errors='coerce', format='%Y-%m-%d %H:%M:%S')
df_claims['IncidentYear'] = df_claims['IncidentDate'].dt.year
df_claims['IntimationDate'] =  pd.to_datetime(df_claims['IntimationDate'],errors='coerce', format='%Y-%m-%d %H:%M:%S')
df_claims['IntimationYear'] = df_claims['IntimationDate'].dt.year


df_claims['VehicleMake'] = np.where(df_claims.VehicleMake.isin(['Suzuki','Toyota','Honda','Kia']) == False, 'Others', df_claims['VehicleMake'])

df_claims.shape

(24438, 83)

In [10]:
df_claims[(df_claims.IntimationYear == 2019) & (df_claims.ClaimStage != 'ClaimClose')].PaidTo.sum() - df_claims[(df_claims.IntimationYear == 2019) & (df_claims.ClaimStage != 'ClaimClose')].SalvageRecovery.sum()

912847682.0899999

In [11]:
df_prem.shape

(60509, 81)

#### Premium Data

In [12]:
df_prem['BillDate'] =  pd.to_datetime(df_prem['BillDate'],errors='coerce', format='%Y-%m-%d %H:%M:%S')

df_prem['Year'] = df_prem['BillDate'].dt.year

df_prem['ExpiryDate'] =  pd.to_datetime(df_prem['ExpiryDate'],errors='coerce', format='%Y-%m-%d %H:%M:%S')

df_prem['EffectiveDate'] =  pd.to_datetime(df_prem['EffectiveDate'],errors='coerce', format='%Y-%m-%d %H:%M:%S')

df_prem['InsuredAmount'] = pd.to_numeric(df_prem['InsuredAmount'],errors='coerce')
df_prem['Adv_Tax'] = pd.to_numeric(df_prem['Adv_Tax'],errors='coerce')

df_prem.VehicleType = df_prem.VehicleType.fillna('None')

#df_prem.drop((df_prem[df_prem['EffectiveDate'].isnull()].index) | (df_prem[df_prem['ExpiryDate'].isnull()].index), inplace = True)



In [13]:
print(df_prem.shape,df_claims.shape)
df_prem.VoucherType.unique()

(60509, 82) (24438, 83)


array(['Renewal', 'Fresh'], dtype=object)

#### Vehicle Data

In [14]:
def float_apply(x):
    try:
        obj = float(x)
        return True
    except ValueError:
        return False
    
def to_make_model(s):
    make  = str(s.split(" ",1)[0])
    model = s[len(make):-4]
    year  = s[-4:]
    return make, model, year

In [15]:
df_prem.dropna(subset=['VehicleMake'],inplace=True)

df_prem['test'] = df_prem.VehicleMake.map(lambda x: to_make_model(x))

new = pd.DataFrame(df_prem['test'].tolist(), index=df_prem.index)
new.columns = ['Make','Model','CarYear']

df_prem = pd.concat([df_prem, new], axis= 1)
df_prem.drop('test', axis=1, inplace=True)
df_prem['Make'] = np.where(df_prem.Make.isin(['Suzuki','Toyota','Honda','Kia']) == False, 'Others', df_prem['Make'])

df_prem.shape

(60509, 85)

In [16]:
#df_prem[(df_prem.Make == 'Honda') & (df_prem['Model'].str.lower().str.contains("city|civic|civi|brv|br-v") == True)].VehicleCategory.value_counts()

#df_prem[(df_prem.Make == 'Toyota') & (df_prem.VehicleCategory == 'Local') ].Model.value_counts() 

#df_prem[df_prem.Model == 'TOYOTA COROLLA GLI 1.3 EFI'] #Gli ECOTEC , TOYOTA COROLLA GLI 1.3 EFI ,Toyota Corolla XLI LE

#df_prem.Make.value_counts()
#df_prem['VehicleCategory_bk'] = df_prem.VehicleCategory
#df_prem.VehicleCategory = 'Empty'
df_prem.VehicleCategory.value_counts()
df_prem.Make.value_counts()
df_prem[(df_prem.Make == 'Honda') & (df_prem.VehicleCategory == 'Local')].VehicleGrossAmount.sum()
#df_prem[df_prem.VehicleCategory == 'Others'].VehicleGrossAmount.sum()

Local       46753
Imported     9381
Others       3907
Name: VehicleCategory, dtype: int64

Toyota    20732
Suzuki    20166
Honda     13582
Others     5744
Kia         285
Name: Make, dtype: int64

461565767.0

In [17]:

# df_prem.VehicleCategory = 'Empty'

# df_prem.VehicleCategory =  np.where((df_prem.Make == 'Honda') & (df_prem['Model'].str.lower().str.contains("city|civic|civi|brv|br-v") == True), 'Local', df_prem.VehicleCategory)

# df_prem.VehicleCategory =  np.where((df_prem.Make == 'Toyota') & (df_prem['Model'].str.lower().str.contains("vigo|revo|fortun|corolla") == True), 'Local', df_prem.VehicleCategory)

# df_prem.VehicleCategory =  np.where((df_prem.Make == 'Kia') & (df_prem['Model'].str.lower().str.contains("carnival|picanto|sportage") == True), 'Local', df_prem.VehicleCategory)

# df_prem.VehicleCategory =  np.where((df_prem.Make == 'Suzuki') & (df_prem['Model'].str.lower().str.contains("apv|baleno|bolan|ciaz|cultus|khyber|kizashi|liana|margalla|mehran|swift|wagon|alto") == True), 'Local', df_prem.VehicleCategory)

# df_prem.VehicleCategory =  np.where((df_prem['Model'].str.lower().str.contains("mp 900|m 280|power cd|rickshaw|d-max|coure|charade|bravo|jeep|lancer|pajero|potohar|prince|santro|sirus|subaru|sunny|v2|van|ravi") == True), 'Others', df_prem.VehicleCategory)

# df_prem.VehicleCategory =  np.where((df_prem['VehicleType'].str.lower().str.contains("commercial|motorcycle|none") == True), 'Others', df_prem.VehicleCategory)

# df_prem.VehicleCategory =  np.where((df_prem.VehicleCategory == 'Empty'), 'Imported', df_prem.VehicleCategory)

df_prem['Make'] = np.where(df_prem.VehicleCategory == 'Imported', 'Imported', df_prem.Make)
#df_prem['Make'] = np.where(df_prem.VehicleCategory == 'Others', 'Others', df_prem.Make)

In [18]:
df_claims.head(2)

,BranchName,ClaimStage,ClientWise,LeadSource,CF_Id,COB_ID,isLeader,sm_ID,Activity_Id,PackageName,ClaimNo,IntimationDate,IncidentCity,IncidentDate,ContactName,NIC_No,Country,City,Area,Passport_No,SalesFormNo,BusinessType,ExpiryDate,EffectiveDate,InsuredAmount,GrossAmount,NetAmount,CircumstancesOfLoss,PaymentAmount,InvoiceAmount,WHTAmount,ClaimTypes,Stage_Id,CloseRemarks,Amount_Reserve,Reserve_GIS,Surveyour,LossController,Workshop,InsuranceType,RI_Amount,RI_Percent,RInsurer,Aging,CI_Amount,CI_Percent,CInsurer,TDI_Percent,TDI_Amount,GroupNo,Agent,PaidTo,VehicleMake,VehicleModel,VehicleColor,VehicleCategory,m_Registration_Year,m_Engine_No,m_Chasis_No,m_VehicleColor_Id,m_Registration_No,ChequeNo,Date,BankName,OutstandingPremium,SalvageRecovery,EstSalvage,Remarks,LossHandler,TaxRegion,claimstatus,WKRegion,WKType,WKMake,PitchRemarks,VehicleType,ClaimRegion,Account_Number,InternalAgent,LeadAgentName,SalesDate,IncidentYear,IntimationYear
0,Faisalabad,Claim Under Process,Corporate,Non 3S,163772,None,None,,57,None,FSD/P/WT/C/163582/0219,2019-02-02 15:28:03.950,Sahiwal,2019-02-02 14:30:00,M/S United Human Resource Services (Pvt) Ltd.,None,Pakistan,Rawalpindi,Dhoke Chaudhrian,None,ISB/P/WT/CUS043263/519097/0518,Renewal,2019-05-19,2018-05-20,"935,100","17,109","21,018","Incident Place & Current Location : Okara City\r\nThe ahead going vehicle suddenly stopped in the middle of the road. In order to avoid any collision, I immediately applied brakes but even then my car could not stop and hit against rear portion of that vehicle. At the same time a motorcyclist coming from behind also hit against left rear portion of my car.","31,375","4,140",nan,Accident,2,,0,,Tanvir-Ur-Rehman Pvt. Ltd.,None,Crystal Car Care--,None,nan,nan,None,444,nan,nan,None,nan,nan,None,,"31,375",Suzuki,Cultus VXR,Silky Silver,Private Motor Car,2015,PKF607672,SF410PK514528,"1,920",ED-867,,,,0,nan,0,,Mr. Rashid Amin,PRA,Claim Under Process,None,None,None,None,Local,CENTRAL,CUS043263,ATIF MEHMOOD,None,2018-05-05 12:11:05.870,2019,2019
1,Faisalabad,Claim Under Process,Corporate,Non 3S,164768,None,None,,57,None,FSD/P/TC/C/164571/0219,2019-02-18 09:59:56.357,Sargodha,2019-01-28 10:00:00,M/S HILTON PHARMA (PVT) LTD,42101-3397628-7,Pakistan,Karachi,Tariq Road,None,KHI/T/TC/CUS045371/010118/0219,Renewal,2019-12-31,2019-01-01,"708,000","18,530","21,124",hit by car,"17,760","7,100",nan,Accident,2,,0,,Tehseen Fazal Associates (Pvt) Ltd..,None,Suzuki Falcon Motors,None,nan,nan,None,428,nan,nan,None,nan,nan,None,,"17,760",Suzuki,Mehran VXR,Graphite Grey,Private Motor Car,2016,667821,1206352,"2,101",BGZ-648,,,,0,nan,0,,Mr. Rashid Amin,PRA,Claim Under Process,Suzuki Falcon Motors,3S,Suzuki,None,Local,CENTRAL,CUS045371,Adrian,Aon Insurance Broker (Pvt) ltd,2019-02-14 12:31:36.470,2019,2019


In [19]:

# df_claims.VehicleType = 'Empty'

# df_claims.VehicleType =  np.where((df_claims.VehicleMake == 'Honda') & (df_claims['VehicleModel'].str.lower().str.contains("city|civic|civi|brv|br-v") == True), 'Local', df_claims.VehicleType)

# df_claims.VehicleType =  np.where((df_claims.VehicleMake == 'Toyota') & (df_claims['VehicleModel'].str.lower().str.contains("vigo|revo|fortun|corolla") == True), 'Local', df_claims.VehicleType)

# df_claims.VehicleType =  np.where((df_claims.VehicleMake == 'Kia') & (df_claims['VehicleModel'].str.lower().str.contains("carnival|picanto|sportage") == True), 'Local', df_claims.VehicleType)

# df_claims.VehicleType =  np.where((df_claims.VehicleMake == 'Suzuki') & (df_claims['VehicleModel'].str.lower().str.contains("apv|baleno|bolan|ciaz|cultus|khyber|kizashi|liana|margalla|mehran|swift|wagon|alto") == True), 'Local', df_claims.VehicleType)

# df_claims.VehicleType =  np.where((df_claims['VehicleModel'].str.lower().str.contains("mp 900|m 280|power cd|rickshaw|d-max|coure|charade|bravo|jeep|lancer|pajero|potohar|prince|santro|sirus|subaru|sunny|v2|van|rabvi") == True), 'Others', df_claims.VehicleType)

# df_claims.VehicleType =  np.where((df_claims['VehicleCategory'].str.lower().str.contains("commercial|motorcycle|none") == True), 'Others', df_claims.VehicleType)

# df_claims.VehicleType =  np.where((df_claims.VehicleType == 'Empty'), 'Imported', df_claims.VehicleType)

df_claims['VehicleMake'] = np.where(df_claims.VehicleType == 'Imported', 'Imported', df_claims.VehicleMake)
#df_claims['VehicleMake'] = np.where(df_claims.VehicleType == 'Others', 'Others', df_claims.VehicleMake)
#df_claims.VehicleType.value_counts()

In [20]:

#df_claims[(df_claims.VehicleMake == 'Honda') & (df_claims.VehicleType == 'Local')].PaidTo.sum()
df_claims.VehicleType.value_counts()
#df_prem[df_prem.VehicleCategory == 'Others'].VehicleGrossAmount.sum()

Local       20617
Imported     3410
Others        321
Name: VehicleType, dtype: int64

#### Claims Data

#### Columns Selection

In [31]:
df_prem[df_prem.VehicleCategory.isna()].Model.value_counts()
df_claims[df_claims.VehicleType.isna()].VehicleModel.value_counts()
df_claims.VehicleType.value_counts()
df_prem.VehicleCategory.value_counts()
#tt[tt.duplicated(['SalesFormNo'], keep=False)].to_csv('duplicate.csv', index = False, header=True)
#df_claims[['ClaimNo','PaidTo','PaymentAmount','CI_Amount','SalvageRecovery','VehicleMake']].to_csv('claims_test.csv', index = False, header=True)
#df_prem.VehicleCategory.isna().any()

 Dayz            112
 MP 900          100
 Power CD-70      82
 M 280            71
 Moov             29
 Mira ES          28
 N Box            24
 Rickshaw         13
 N-Wagon           5
 Kizashi           3
 Dazey             1
Name: Model, dtype: int64

Dayz       32
Mira ES    25
Moov       12
N Box      11
M 280       3
N-Wagon     3
Kizashi     2
MP 900      2
Name: VehicleModel, dtype: int64

Local       20617
Imported     3410
Others        321
Name: VehicleType, dtype: int64

Local       46753
Imported     9381
Others       3907
Name: VehicleCategory, dtype: int64

In [22]:
#df_prem = prem_bk
df_prem['Policy Count'] = df_prem.groupby(['Make','Year','VoucherType','SalesBranch','VehicleCategory','LeadSource'])['SalesFormNo'].transform('nunique')
#df_prem['Make'] = np.where(df_prem.VehicleCategory == 'Imported', 'Imported', df_prem.Make)


premCols = [
         'SalesFormNo',
         'Year',
         'Product',
         'Make',
         'VoucherType',
         'SalesBranch',
         'Account_Name',
         'Account_Number',
         'LeadSource',
         'VehicleCategory',
         'VehicleGrossAmount',
         'Policy Count',
         
]

groupbyCols = [#'SalesFormNo',
               'Year',
               'Make',
               'VoucherType',
               'SalesBranch',
               'LeadSource',
               'VehicleCategory',
               'Policy Count',
               
              ]



dfp_grp = df_prem#[premCols]
dfp_grp = dfp_grp.groupby(groupbyCols)['VehicleGrossAmount'].agg(['sum']).reset_index()
dfp_grp.columns = dfp_grp.columns.get_level_values(0)

dfp_grp.rename(columns={'branch': 'SalesBranch','VoucherType':'BusinessType','LeadSource':'Channel','sum':'Gross Premium','count':'Policy Count'}, inplace=True)


dfp_grp['Gross Premium'].sum()
#df_prem.VehicleGrossAmount.sum()
dfp_grp['Policy Count'].sum()
dfp_grp
   
#dfp_grp[(dfp_grp.SalesBranch == 'Islamabad')  & (dfp_grp.Channel == '3S') & (dfp_grp.BusinessType == 'Fresh') & (dfp_grp.Make == 'Honda') & (dfp_grp.VehicleCategory == 'Imported')]


2098771333.0

49359.0

,Year,Make,BusinessType,SalesBranch,Channel,VehicleCategory,Policy Count,Gross Premium
0,2019,Honda,Fresh,Faisalabad,3S,Local,71,"3,427,444"
1,2019,Honda,Fresh,Faisalabad,Direct,Local,28,"1,903,308"
2,2019,Honda,Fresh,Faisalabad,Non 3S,Local,117,"5,837,984"
3,2019,Honda,Fresh,Faisalabad,Non 3S,Others,3,"4,918"
4,2019,Honda,Fresh,Gujranwala,3S,Local,7,"301,344"
...,...,...,...,...,...,...,...,...
404,2019,Toyota,Renewal,Peshawar,3S,Local,36,"1,422,573"
405,2019,Toyota,Renewal,Peshawar,CallCentre,Local,2,"108,984"
406,2019,Toyota,Renewal,Peshawar,Direct,Others,1,"65,616"
407,2019,Toyota,Renewal,Peshawar,Non 3S,Local,254,"11,382,194"


In [545]:
def trim_all_columns(df):
    """
    Trim whitespace from ends of each value across all series in dataframe
    """
    trim_strings = lambda x: x.strip() if isinstance(x, str) else x
    return df.applymap(trim_strings)

#prem_bk = df_prem.copy()
#prem_bk[['SalesFormNo','Make','Year','VoucherType','SalesBranch','VehicleCategory','LeadSource']] = prem_bk[['SalesFormNo','Make','Year','VoucherType','SalesBranch','VehicleCategory','LeadSource']].astype(str)
#prem_bk = trim_all_columns(prem_bk)
#duplicats = prem_bk.groupby(['SalesFormNo','Make'])['SalesFormNo'].size()
#duplicats[duplicats > 1]
dfp_grp[(dfp_grp.Make == 'Honda') & (dfc_grp.VehicleCategory == 'Imported')]['Gross Premium'].sum()

/home/data/anaconda3/envs/TestEnv/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



0.0

#### Join Prem & Vehcile Data

In [486]:
#df_claims.fillna(0,inplace=True) 
#df_claims['VehicleMake'] = np.where(df_claims.VehicleMake.isin(['Suzuki','Toyota','Honda','Kia']) == False, 'Others', df_claims['VehicleMake'])    
#df_claims.VehicleMake.unique()
#df_claims[df_claims.VehicleType != 0]

###FILL NA VEHICLE TYPE TO OTHERS

In [23]:
df_claims.VehicleType.isna().any()

True

In [24]:
df_claims[df_claims.IntimationYear == 2019].shape


(24438, 83)

#### Join Prem,Vehicle & Claim Amount Data

In [25]:
#SalesFormNo',
df_claims.Amount_Reserve.fillna(0,inplace=True)
df_claims.PaidTo.fillna(0,inplace=True)
df_claims.SalvageRecovery.fillna(0,inplace=True)
df_claims.PaymentAmount.fillna(0,inplace=True)
df_claims.CI_Amount.fillna(0,inplace=True)
df_claims.CI_Percent.fillna(0,inplace=True)

df_claims.CI_Amount = (df_claims.Amount_Reserve + df_claims.PaidTo) * (df_claims.CI_Percent/100)

claimCols = [
             'IntimationYear',
             'PaidTo',
             'SalvageRecovery',
             'VehicleMake',
             'ClaimTypes',
             'BusinessType',
             'BranchName',
             'LeadSource',
             'VehicleType',
             'CI_Amount',
             'Amount_Reserve',   
             'ClaimNo'
         
]

groupbyCols = [
               'IntimationYear',
               'VehicleMake',
                 'ClaimTypes',
                 'BusinessType',
                 'BranchName',
                 'LeadSource',
                 'VehicleType',
                 'PaidTo',
                 'SalvageRecovery',
                 'CI_Amount',
                 'Amount_Reserve', 
                 'ClaimNo'
               
              ]

#df_claims[(df_claims.IntimationYear == 2019) & (df_claims.ClaimStage != 'ClaimClose')].PaidTo.sum() - df_claims[(df_claims.IntimationYear == 2019) & (df_claims.ClaimStage != 'ClaimClose')].SalvageRecovery.sum()


df_claims['VehicleType'] = np.where(df_claims.VehicleType == 0, 'Others', df_claims.VehicleType)
#df_claims['VehicleMake'] = np.where(df_claims.VehicleType == 'Imported', 'Imported', df_claims.VehicleMake)

dfc_grp = df_claims[claimCols]#[df_claims.ClaimStage != 'ClaimClose'][claimCols]

dfc_grp = dfc_grp.groupby(groupbyCols)['PaidTo'].agg(['sum','count']).reset_index()
dfc_grp['sum'] =(dfc_grp.Amount_Reserve + dfc_grp.PaidTo - dfc_grp.CI_Amount) - dfc_grp.SalvageRecovery
dfc_grp.rename(columns={'BusinessType':'VoucherType','BranchName':'SalesBranch'}, inplace=True)
dfc_grp['ClaimTypes'] = np.where(dfc_grp.ClaimTypes.str.strip() == 'Snatch', 'Theft', dfc_grp.ClaimTypes)
dfc_grp['ClaimTypes'] = dfc_grp['ClaimTypes'].str.strip()

dfc_grp.rename(columns={'IntimationYear':'Year', 'VehicleMake':'Make', 'VehicleType': 'VehicleCategory','VoucherType':'BusinessType','LeadSource':'Channel','sum':'TotalClaimed','count':'Claim Count'}, inplace=True)
dfc_grp[dfc_grp.Year == 2019].TotalClaimed.sum()
dfc_grp['Claim Count'].sum()
dfc_grp#[dfc_grp.Year == 2019]
#dfc_grp[(dfc_grp.SalesBranch == 'Karachi')  & (dfc_grp.Channel == '3S') & (dfc_grp.BusinessType == 'Fresh') & (dfc_grp.Make == 'Others') & (dfc_grp.VehicleCategory == 'Imported')]



987911617.799

24348

,Year,Make,ClaimTypes,BusinessType,SalesBranch,Channel,VehicleCategory,PaidTo,SalvageRecovery,CI_Amount,Amount_Reserve,ClaimNo,TotalClaimed,Claim Count
0,2019,Honda,Accessory Theft,Fresh,Faisalabad,3S,Local,"57,400",0,0,0,FSD/P/DIB/C/164200/0219,"57,400",1
1,2019,Honda,Accessory Theft,Fresh,Hyderabad,3S,Local,"6,310",0,0,0,HYD/P/WT/C/168866/0419,"6,310",1
2,2019,Honda,Accessory Theft,Fresh,Islamabad,Digital,Local,0,0,0,0,ISB/P/WT/C/179704/0919,0,1
3,2019,Honda,Accessory Theft,Fresh,Islamabad,Non 3S,Local,"134,391",0,0,0,ISB/P/ABB/C/169843/0419,"134,391",1
4,2019,Honda,Accessory Theft,Fresh,Karachi,3S,Local,0,0,0,0,KHI/P/DIB/C/162338/0119,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24343,2019,Toyota,Total-Loss,Renewal,Karachi,Non 3S,Local,"1,772,100",0,0,"6,000",KHI/P/JSB/C/172233/0619,"1,778,100",1
24344,2019,Toyota,Total-Loss,Renewal,Karachi,Non 3S,Local,"1,825,000","1,460,000",0,0,KHI/P/TC/C/162091/0119,"365,000",1
24345,2019,Toyota,Total-Loss,Renewal,Lahore,CallCentre,Local,"1,605,650","1,405,000",0,0,LHR/P/WT/C/176515/0819,"200,650",1
24346,2019,Toyota,Total-Loss,Renewal,Lahore,Non 3S,Local,"1,025,650","900,000",0,0,LHR/P/ABB/C/172579/0619,"125,650",1


In [26]:
tdf_prem = dfp_grp.groupby(['Year','Make'])['Gross Premium','Policy Count'].agg(['sum']).reset_index()
tdf_claim = dfc_grp.groupby(['Year','Make'])['TotalClaimed','Claim Count'].agg(['sum']).reset_index()

tdf_prem.columns = tdf_prem.columns.get_level_values(0)
tdf_claim.columns = tdf_claim.columns.get_level_values(0)
tdf_prem['Gross Premium'].sum()

tdf_claim.TotalClaimed.sum()
tdf_prem
tdf_claim
tdf = tdf_prem.merge(tdf_claim,on=['Year','Make'],how="left")
tdf 

# tdf_prem.dtypes
# tdf_claim.dtypes

/home/data/anaconda3/envs/TestEnv/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.

/home/data/anaconda3/envs/TestEnv/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



2098771333.0

987911617.7990001

,Year,Make,Gross Premium,Policy Count
0,2019,Honda,"464,875,581","11,219"
1,2019,Imported,"495,380,042","7,878"
2,2019,Kia,"16,402,534",175
3,2019,Others,"106,887,669","1,510"
4,2019,Suzuki,"375,553,857","15,311"
5,2019,Toyota,"639,671,650","13,266"


,Year,Make,TotalClaimed,Claim Count
0,2019,Honda,"211,089,254",5768
1,2019,Imported,"188,972,671",3410
2,2019,Kia,"1,793,667",12
3,2019,Others,"31,779,911",248
4,2019,Suzuki,"224,042,135",8208
5,2019,Toyota,"330,233,980",6702


,Year,Make,Gross Premium,Policy Count,TotalClaimed,Claim Count
0,2019,Honda,"464,875,581","11,219","211,089,254",5768
1,2019,Imported,"495,380,042","7,878","188,972,671",3410
2,2019,Kia,"16,402,534",175,"1,793,667",12
3,2019,Others,"106,887,669","1,510","31,779,911",248
4,2019,Suzuki,"375,553,857","15,311","224,042,135",8208
5,2019,Toyota,"639,671,650","13,266","330,233,980",6702


In [27]:
import humanize as hm
print('Data from 2019 to 2020 Feb 27th')
dict4Model = {}

# dict4Model['df_prem'] = df_premdf_prem
# dict4Model['df_vehicle'] = df_vehicle
# dict4Model['dfc_grp'] = dfc_grp

# dict4Model['data'] = tdf

businessTypeList = dfp_grp.BusinessType.unique().tolist()
businessTypeList.insert(0, 'All')
# dict4Model['BusinessTypes'] = businessTypeList

branchList = dfp_grp.SalesBranch.unique().tolist()
branchList.insert(0, 'All')
# dict4Model['Branches'] = branchList

claimTypeList = dfc_grp.ClaimTypes.str.strip().unique().tolist()
claimTypeList.insert(0, 'All')
# dict4Model['ClaimTypes'] = claimTypeList

# claimStageList = dfc_grp.ClaimStage.str.strip().unique().tolist()
# claimStageList.insert(0, 'All')
# dict4Model['ClaimTypes'] = claimTypeList

channelList = dfp_grp.Channel.str.strip().unique().tolist()
channelList.insert(0, 'All')
# dict4Model['ChannelTypes'] = channelList

vehicleMakeList = dfp_grp.Make.str.strip().unique().tolist()
vehicleMakeList.insert(0, 'All')

yearList = dfp_grp.Year.unique().tolist()
yearList.insert(2, 'All')

# import pickle
# with open('/home/data/Projects/MISLossProfile/pickle/modelData.pickle', 'wb') as f:
#     pickle.dump(dict4Model, f)

def MISReport(BusinessType,SalesBranch,Channel,Make,Year,ClaimTypes,High_Claim_Details,Misc_Report,premData,claimData):
    
    if (High_Claim_Details == False) & (Misc_Report == False):
        
        dataKVP = {'BusinessType': BusinessType, 'SalesBranch': SalesBranch, 'Channel': Channel, 'Year':Year, 'Make': Make, 'ClaimTypes':ClaimTypes}
        
        filter_str = buildFilter(dataKVP)
        
        if filter_str != -1:  ##if any filter applied 
            
            if "ClaimTypes" in filter_str: ## if ClaimFilter then dont use with PremData
                if len(filter_str.rsplit('and',1)) > 1: ## if len is 2 means other filters also selected with claimType
                    dfp_grp = premData.query(filter_str.rsplit('and',1)[0])
                    dfc_grp = claimData.query(filter_str)
                else:
                    dfp_grp = premData
                    dfc_grp = claimData.query(filter_str)
            else:
                dfp_grp = premData.query(filter_str)
                dfc_grp = claimData.query(filter_str)
        else:
            dfp_grp = premData
            dfc_grp = claimData
        
        
        
        tdf_prem = dfp_grp.groupby(['Year','Make'])['Gross Premium','Policy Count'].agg(['sum']).reset_index()
        tdf_claim = dfc_grp.groupby(['Year','Make'])['TotalClaimed','Claim Count'].agg(['sum']).reset_index()

        tdf_prem.columns = tdf_prem.columns.get_level_values(0)
        tdf_claim.columns = tdf_claim.columns.get_level_values(0)
        

        tdf = tdf_prem.merge(tdf_claim.drop_duplicates(),on=['Year','Make'],how="left")

        
        tdf = pd.pivot_table(tdf,index='Make',values=[ 'Gross Premium','Policy Count','TotalClaimed','Claim Count'],aggfunc=np.sum, margins=True)

        pd.options.display.float_format = '{:,.2f}'.format

        #tdf=tdf.stack(level=1)
    
        if tdf.shape[0] > 0:
            tdf['Claim Ratio%'] = (tdf['TotalClaimed'].astype(float) / tdf['Gross Premium'].astype(float)) * 100
            tdf['Claim Freq%'] = (tdf['Claim Count'].astype(float) / tdf['Policy Count'].astype(float)) * 100
        else:
            print('No Records Found')
            return None

        #tdf=tdf.unstack()
        #tdf.reorder_levels([1,0],axis=1).sort_index(axis=1)
        tdf.fillna(0,inplace=True)
       
        tdf = tdf.astype(int)
        #tdf = tdf.applymap(hm.intword)
        return tdf
    elif High_Claim_Details == True :
        return dfc_grp[(dfc_grp.ClaimAmount >= 500000)][dfc_grp.columns.difference(['CircumstancesOfLoss'])].applymap(hm.intword)
    else : 
        
        tdf = pd.read_csv('~/Projects/InsuranceAnalysis/data/CLA_MotorPaidReport_2018Onwards.csv',sep='|',engine='python')
        tdf['RecieptDate'] =  pd.to_datetime(tdf['RecieptDate'],errors='coerce', format='%Y-%m-%d %H:%M:%S')
        tdf = tdf[tdf.ClaimStage == 'Final Cost Estimation'].groupby([tdf.RecieptDate.dt.year,'PaymentHead'])['PaidAmount'].agg(['sum','count']).reset_index()
        tdf.columns = ['RecieptDate','PaymentHead', 'Calim Sum','No of Claims']
        tdf = tdf.pivot(index='PaymentHead',columns='RecieptDate',values=[ 'No of Claims','Claim Sum'])
        tdf = tdf.stack(level=1)
        tdf = tdf.astype(int)
        tdf = tdf.applymap(hm.intword)
        
        return tdf

    
def buildFilter(dataKVP):
    s = ''
    
    for k,v in dataKVP.items():
        if v != 'All':
            s += k + '==' + f'"{v}"' + ' and '
    
    if ('and' in s):
        print(s[:s.rindex('and')])
        return s[:s.rindex('and')]
    else:
        return -1
    
interact(MISReport,
         BusinessType=businessTypeList,
         SalesBranch=branchList,
         ClaimTypes=claimTypeList,
         Channel=channelList,
         Make=vehicleMakeList,
         Year=yearList,
         High_Claim_Details=False,Misc_Report=False,
         premData=fixed(dfp_grp),claimData=fixed(dfc_grp))

Data from 2019 to 2020 Feb 27th


interactive(children=(Dropdown(description='BusinessType', options=('All', 'Fresh', 'Renewal'), value='All'), …

<function __main__.MISReport(BusinessType, SalesBranch, Channel, Make, Year, ClaimTypes, High_Claim_Details, Misc_Report, premData, claimData)>

In [232]:
from datetime import datetime
from datetime import timedelta
today = datetime.today()
yesterday = today - timedelta(days=364 + datetime.now().timetuple().tm_yday)
print(today.now().date())
print()
print(yesterday.date())


2020-03-04

2019-01-01


In [233]:
type(yesterday)

datetime.datetime